In [1]:
import numpy as np
import matplotlib.pylab as plt
import scipy as sp
from scipy import constants as const
from scipy.integrate import simps
from scipy.optimize import minimize
from findiff import FinDiff, coefficients, Coefficient

In [2]:
def create_weights(n_hidden, n_neurons):
    weights_1 = np.random.rand(1, n_neurons)
    weights_last = np.random.rand(n_neurons, 1)
    weights_hidden = [np.random.rand(n_neurons, n_neurons) for i in range(n_hidden - 1)]
    weights_hidden.insert(0, weights_1)
    weights_hidden.append(weights_last)
    return weights_hidden
def relu(x):
    return np.maximum(np.zeros(x.shape), x)
def forward(weights_hidden, x):
    weights_hidedden = un_flatten(weights_hidden)
    for weight_array in weights_hidden:
        x = np.dot(x, weight_array)
        x = relu(x)
    return x

In [3]:
def flatten(c):
    c_flat = []
    shapes = []
    lengths = []
    for i in c:
        shapes.append(np.shape(i))
        c_flat += list(i.flatten())
        lengths.append(i.size)
    return c_flat, shapes, lengths

In [4]:
def un_flatten(c):
    c_flat, shapes, lengths = flatten(c)
    c_unflat = []
    for i in range(len(shapes)):
        c_unflat.append(np.reshape(c_flat[:lengths[i]], shapes[i]))
        del c_flat[:lengths[i]]
    return c_unflat

In [5]:
def psi_n(c, x):
    def integral(c, x):
        integral = (forward(c, x) ** 2)
        return integral
    A = (simps(integral(c, x), x)) ** -0.5
    psi_n = A * forward(c, x)
    return psi_n

In [6]:
def d2psi_n_dx2(c, x):
    dx = x[1] - x[0]
    d2_dx2 = FinDiff(0, dx, 2)
    d2psi_n_dx2 = d2_dx2(psi_n(c, x))
    return d2psi_n_dx2

In [7]:
def V(c, x, L):
    V = []
    for i in x:
        if 0 < i and i < L:
            V.append(0)
        else:
            V.append(50)
    return V

In [8]:
# def E(c, x, L):
#     E = []
#     for i in x:
#         E.append((const.hbar ** 2) / (2 * const.m_e)) * (d2psi_n_dx2(c, i) / psi_n(c, i)) + V(c, i, L)
#     # I might need a negative above
#     return np.array(E)

In [9]:
def E(c, x, L):
    E = ((const.hbar ** 2) / (2 * const.m_e)) * (d2psi_n_dx2(c, x) / psi_n(c, x)) + V(c, x, L)
    # I might need a negative above
    return E

In [10]:
def energy_function(c_flat):
    c = un_flatten(c_flat)
    true_energy = (const.h ** 2) / (8 * const.m_e * L ** 2)
    return E(c, x, L)[0] - true_energy

In [11]:
L = 10
x = np.linspace(1, L, 1000)

true_energy = (const.h ** 2) / (8 * const.m_e * L ** 2)

In [12]:
c = create_weights(2, 10)
forward(c, np.array([1]))

array([13.28761542])

In [13]:
c = flatten(c)[0]

res = minimize(energy_function, c)

c = un_flatten(res.x)

energy = E(c, x, L)

# Print the ground state energy and corresponding coefficients
print("True ground state energy:", true_energy)
print("Ground state energy:", np.min(energy))
print("Optimized coefficients:", res.x)

True ground state energy: 6.02466740223757e-40
Ground state energy: -3.0061490483422503e-49
Optimized coefficients: [0.79442199 0.90391037 0.40724573 0.32392165 0.42624829 0.86800278
 0.69623461 0.08155885 0.46674122 0.37803283 0.47056842 0.98504499
 0.10510136 0.12296161 0.71350129 0.34363861 0.97176464 0.01226253
 0.07014306 0.80288624 0.04091755 0.86557209 0.25857097 0.72016919
 0.62499221 0.50954717 0.45568412 0.33377546 0.60949925 0.66950014
 0.89238456 0.49155753 0.73967679 0.56275111 0.19758473 0.64335772
 0.27805665 0.24929365 0.46792517 0.67447739 0.8650326  0.05791573
 0.04941626 0.2730016  0.27092819 0.62905241 0.41068332 0.37926154
 0.74091092 0.00220127 0.86486819 0.78691385 0.60797625 0.5227553
 0.06197004 0.74072691 0.2058467  0.26806465 0.63312126 0.96239233
 0.11009294 0.09435207 0.53144284 0.20679458 0.80182894 0.51903813
 0.48292105 0.53472944 0.11965915 0.30064555 0.0210381  0.28030018
 0.62475608 0.11705264 0.22510036 0.58862963 0.99530734 0.88659569
 0.71816858 0.